In [1]:
import torch
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")
else:
    print('can use GPU')
    
mps_device = torch.device("mps")

can use GPU


In [2]:
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root = 'data',
    train=True,
    download=False,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=False,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork,self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    

model = NeuralNetwork()
mps_model = NeuralNetwork().to(mps_device)

In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [4]:
loss_fn = nn.CrossEntropyLoss()

In [5]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [6]:
def train_loop(device, dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        if device == 'GPU':
            X = X.to(mps_device)
            y = y.to(mps_device)
        pred = model(X)
        loss = loss_fn(pred,y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch%100 == 0:
            loss, current = loss.item(), batch*len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
    
def test_loop(device, dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0,0
    
    with torch.no_grad():
        for X, y in dataloader:
            if device == 'GPU':
                X = X.to(mps_device)
                y = y.to(mps_device)
            pred = model(X)
            test_loss += loss_fn(pred,y).item()
            correct +=(pred.argmax(1) == y).type(torch.float).sum().item()
        
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n----------------------------")
    train_loop('CPU',train_dataloader, model, loss_fn, optimizer)
    test_loop('CPU',test_dataloader, model, loss_fn)
print("Done")

Epoch 1
----------------------------
loss: 2.306662 [    0/60000]
loss: 2.288401 [ 6400/60000]
loss: 2.274842 [12800/60000]
loss: 2.264933 [19200/60000]
loss: 2.249988 [25600/60000]
loss: 2.236196 [32000/60000]
loss: 2.237937 [38400/60000]
loss: 2.214744 [44800/60000]
loss: 2.204525 [51200/60000]
loss: 2.171767 [57600/60000]
Test Error: 
 Accuracy: 44.8%, Avg loss: 2.170447

Epoch 2
----------------------------
loss: 2.189184 [    0/60000]
loss: 2.167546 [ 6400/60000]
loss: 2.122059 [12800/60000]
loss: 2.127918 [19200/60000]
loss: 2.077368 [25600/60000]
loss: 2.036352 [32000/60000]
loss: 2.052095 [38400/60000]
loss: 1.991274 [44800/60000]
loss: 1.990990 [51200/60000]
loss: 1.906897 [57600/60000]
Test Error: 
 Accuracy: 56.3%, Avg loss: 1.916585

Epoch 3
----------------------------
loss: 1.958540 [    0/60000]
loss: 1.913659 [ 6400/60000]
loss: 1.816010 [12800/60000]
loss: 1.842449 [19200/60000]
loss: 1.728814 [25600/60000]
loss: 1.690781 [32000/60000]
loss: 1.698013 [38400/60000]
loss

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(mps_model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n----------------------------")
    train_loop('GPU',train_dataloader, mps_model, loss_fn, optimizer)
    test_loop('GPU',test_dataloader, mps_model, loss_fn)
print("Done")

Epoch 1
----------------------------
loss: 2.293835 [    0/60000]
loss: 2.285913 [ 6400/60000]
loss: 2.264354 [12800/60000]
loss: 2.262019 [19200/60000]
loss: 2.243510 [25600/60000]
loss: 2.218347 [32000/60000]
loss: 2.223123 [38400/60000]
loss: 2.188939 [44800/60000]
loss: 2.178814 [51200/60000]
loss: 2.145448 [57600/60000]
Test Error: 
 Accuracy: 47.2%, Avg loss: 2.147273

Epoch 2
----------------------------
loss: 2.153404 [    0/60000]
loss: 2.148616 [ 6400/60000]
loss: 2.084133 [12800/60000]
loss: 2.100722 [19200/60000]
loss: 2.052441 [25600/60000]
loss: 1.988342 [32000/60000]
loss: 2.015006 [38400/60000]
loss: 1.933896 [44800/60000]
loss: 1.933100 [51200/60000]
loss: 1.848238 [57600/60000]
Test Error: 
 Accuracy: 56.0%, Avg loss: 1.863882

Epoch 3
----------------------------
loss: 1.897423 [    0/60000]
loss: 1.871326 [ 6400/60000]
loss: 1.745223 [12800/60000]
loss: 1.781897 [19200/60000]
loss: 1.683750 [25600/60000]
loss: 1.633418 [32000/60000]
loss: 1.649271 [38400/60000]
loss